---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [3]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [5]:
def answer_one():
    records = spam_data['target'].value_counts()
    print(records)
    spam_len = [len(j) for i, j in enumerate(spam_data['text']) if spam_data['target'][i] == 0]
    print(len(spam_len))
    
    return (records[1]/len(spam_data['target'])) * 100

In [55]:
answer_one()

0    4825
1     747
Name: target, dtype: int64
4825


13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(X_train)

def answer_two():
    
    words = vect.get_feature_names()
    
    return max(words, key=len)

In [37]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    X_train_vectorized = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [55]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    tfidf_array = X_train_vectorized.max(0).toarray()[0]
   
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    
    sorted_tfidf_values = tfidf_array[sorted_tfidf_index]
    
    s = pd.Series(sorted_tfidf_values[0:20],index=feature_names[sorted_tfidf_index[:20]])
    
    res = s[np.lexsort((s.index, s.values))]

    d = pd.Series(sorted_tfidf_values[:-21:-1],index=feature_names[sorted_tfidf_index[:-21:-1]])
    
    res1 = d[np.lexsort((d.index, -d.values))]
        
    
    return (res, res1)

In [31]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [9]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)

    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)

    predictions = model.predict(vect.transform(X_test))

    auc_score = roc_auc_score(y_test, predictions)
    
    
    return auc_score

In [33]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [10]:
def answer_six():
    spam_len = [len(j) for i, j in enumerate(spam_data['text']) if spam_data['target'][i] == 1]
    avg_spam = sum(spam_len)/len(spam_len)
    not_spam_len = [len(j) for i, j in enumerate(spam_data['text']) if spam_data['target'][i] == 0]
    avg_not_spam = sum(not_spam_len)/len(not_spam_len)
    
    
    return (avg_not_spam, avg_spam)

In [63]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [11]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized1 = vect.transform(X_train)
    X_train_vectorized = add_feature(X_train_vectorized1, X_train.str.len())
    
    model = SVC(C=10000)
    model.fit(X_train_vectorized, y_train)
    
    test = vect.transform(X_test)
    X_test_vectorized = add_feature(test, X_test.str.len())
    
    predictions = model.predict(X_test_vectorized)

    auc_score = roc_auc_score(y_test, predictions)
    
    
    return auc_score

In [54]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [13]:
def answer_eight():
    spam_len = [sum(c.isdigit() for c in j) for i, j in enumerate(spam_data['text']) if spam_data['target'][i] == 1]
    avg_spam = sum(spam_len)/len(spam_len)
    not_spam_len = [sum(c.isdigit() for c in j) for i, j in enumerate(spam_data['text']) if spam_data['target'][i] == 0]
    avg_not_spam = sum(not_spam_len)/len(not_spam_len)
    
    
    return (avg_not_spam, avg_spam)

In [68]:
answer_eight()

(15.759036144578314, 0.2992746113989637)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [71]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    digits_len_train = [sum(c.isdigit() for c in i) for i in X_train] 
    digits_len_test = [sum(c.isdigit() for c in i) for i in X_test] 
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized1 = vect.transform(X_train)
    X_train_vectorized = add_feature(X_train_vectorized1, X_train.str.len())
    X_train_vectorized = add_feature(X_train_vectorized1, digits_len_train)
    
    model = LogisticRegression(C=100)
    model.fit(X_train_vectorized, y_train)
    
    test = vect.transform(X_test)
    X_test_vectorized = add_feature(test, X_test.str.len())
    X_test_vectorized = add_feature(test, digits_len_test)
    
    
    predictions = model.predict(X_test_vectorized)

    auc_score = roc_auc_score(y_test, predictions)
    
    
    return auc_score

In [72]:
answer_nine()

0.96787090640544626

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [15]:
import re
def answer_ten():
    spam_len = [len(j)-len(re.findall('[\w]', j)) for i, j in enumerate(spam_data['text']) if spam_data['target'][i] == 1]
    avg_spam = sum(spam_len)/len(spam_len)
    not_spam_len = [len(j)-len(re.findall('[\w]', j)) for i, j in enumerate(spam_data['text']) if spam_data['target'][i] == 0]
    avg_not_spam = sum(not_spam_len)/len(not_spam_len)
    
    
    
    return (avg_not_spam, avg_spam)

In [84]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
def answer_eleven():
    digits_len_train = [sum(c.isdigit() for c in i) for i in X_train] 
    non_words_len_train = [len(i)-len(re.findall('[\w]', i)) for i in X_train] 
    digits_len_test = [sum(c.isdigit() for c in i) for i in X_test] 
    non_words_len_test = [len(i)-len(re.findall('[\w]', i)) for i in X_test]
    
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vectorized1 = vect.transform(X_train)
    print(X_train_vectorized1[0])
    X_train_vectorized = add_feature(X_train_vectorized1, X_train.str.len())
    print(X_train_vectorized[0])
    X_train_vectorized = add_feature(X_train_vectorized, digits_len_train)
    print(X_train_vectorized[0])
    X_train_vectorized = add_feature(X_train_vectorized, non_words_len_train)
    print(X_train_vectorized[0])
    
    model = LogisticRegression(C=100)
    model.fit(X_train_vectorized, y_train)
    
    test = vect.transform(X_test)
    X_test_vectorized = add_feature(test, X_test.str.len())
    X_test_vectorized = add_feature(X_test_vectorized, digits_len_test)
    X_test_vectorized = add_feature(X_test_vectorized, non_words_len_test)
    
    predictions = model.predict(X_test_vectorized)

    auc_score = roc_auc_score(y_test, predictions)
    
    feature_names = np.array(vect.get_feature_names())
#     print(feature_names)
    
#     feature_names = np.append(feature_names, ["length_of_doc", "digit_count", "non_word_char_count"])
#     print([X_train.str.len(),digits_len_train,non_words_len_train])
    feature_names = np.array(vect.get_feature_names() + ["length_of_doc", "digit_count", "non_word_char_count"])
    
#     feature_names += [ X_train.str.len(), digits_len_train, non_words_len_train]

    sorted_coef_index = model.coef_[0].argsort()

    smallest_coefs_list = feature_names[sorted_coef_index[:10]]
    largest_coefs_list = feature_names[sorted_coef_index[:-11:-1]]
#     largest_coefs_list = np.fliplr([largest_coefs_list])[0]
    x = pd.Series(smallest_coefs_list) 
    y = pd.Series(largest_coefs_list) 
    return (auc_score, x, y)

In [76]:
answer_eleven()

  (0, 764)	1
  (0, 882)	1
  (0, 891)	1
  (0, 892)	1
  (0, 1310)	2
  (0, 1311)	1
  (0, 1312)	1
  (0, 1315)	1
  (0, 1316)	1
  (0, 1866)	1
  (0, 1872)	1
  (0, 1874)	1
  (0, 1875)	1
  (0, 2457)	1
  (0, 2482)	1
  (0, 2504)	1
  (0, 2505)	1
  (0, 2748)	1
  (0, 2814)	1
  (0, 2817)	1
  (0, 2818)	1
  (0, 2893)	1
  (0, 2894)	1
  (0, 2907)	1
  (0, 2934)	1
  :	:
  (0, 11516)	1
  (0, 11896)	1
  (0, 11897)	1
  (0, 12113)	1
  (0, 12114)	1
  (0, 12301)	1
  (0, 13256)	1
  (0, 13300)	1
  (0, 13301)	1
  (0, 14215)	1
  (0, 14367)	1
  (0, 14464)	1
  (0, 14465)	1
  (0, 14466)	1
  (0, 14912)	1
  (0, 15706)	1
  (0, 15713)	1
  (0, 15714)	1
  (0, 15715)	1
  (0, 15881)	1
  (0, 15949)	1
  (0, 15950)	1
  (0, 16094)	1
  (0, 16104)	1
  (0, 16105)	1
  (0, 764)	1
  (0, 882)	1
  (0, 891)	1
  (0, 892)	1
  (0, 1310)	2
  (0, 1311)	1
  (0, 1312)	1
  (0, 1315)	1
  (0, 1316)	1
  (0, 1866)	1
  (0, 1872)	1
  (0, 1874)	1
  (0, 1875)	1
  (0, 2457)	1
  (0, 2482)	1
  (0, 2504)	1
  (0, 2505)	1
  (0, 2748)	1
  (0, 2814)	1
  (0, 2817)

(0.97885931107074342, 0     . 
 1     ..
 2     ? 
 3      i
 4      y
 5     go
 6     :)
 7      h
 8     go
 9      m
 dtype: object, 0    digit_count
 1             ne
 2             ia
 3             co
 4             xt
 5             ch
 6            mob
 7              x
 8             ww
 9             ar
 dtype: object)